# 推进速度电位器设定值

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot
# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

from preprocessing import *

data_path = Path('tbmData/data')
fn_txt = sorted(data_path.glob('*.txt')); print(fn_txt[:3])
fn_cycles = Path('tbmData/cycles1')

num_cycles = 3481
valid_ratio = 0.2
train_ratio = 1 - valid_ratio
valid_idx = np.arange(int(num_cycles * valid_ratio))

mulr = 50
bs = int(num_cycles * train_ratio) 

cont_names, dep_var = ['推进速度', '主驱动1#电机扭矩', '刀盘扭矩', '刀盘转速', '主液压油箱温度'], '推进速度电位器设定值'

In [ ]:
# with concurrent.futures.ThreadPoolExecutor() as e:
#     cycles = tqdm_notebook(e.map(read_feather_fn, sorted(fn_cycles.glob('cycle*'))[:num_cycles]), desc='read_dataframe', total=num_cycles)
#     idx = list(tqdm_notebook(e.map(beginning_index, [o.iloc[:500] for o in cycles]), desc='beginning', total=num_cycles))
# #     cont = [o.loc[:,cont_names].iloc[i:i+30] for i, o in zip(idx, cycles)]
#     cont = [o.iloc[:,3:].iloc[i:i+30] for i, o in zip(idx, cycles)]

In [ ]:
fns_feathers = sorted(fn_cycles.glob('cycle*'))[:num_cycles]
fmtr = DataFormatter(cycle_feathers=fns_feathers)
cycles = fmtr.cycles
idx = [beginning_index(o.iloc[:500]) for o in tqdm_notebook(cycles)]

In [ ]:
def tile_with_noise(cycles, noise_size=(-2, 5)):
    noise = (np.random.random(mulr) * (noise_size[1]-noise_size[0]+1)).astype('uint8') + noise_size[0]
    df_conts = [o.loc[:,cont_names].iloc[i+n:i+30+n] for n in tqdm_notebook(noise)
                                                    for i, o in zip(idx, cycles)]
    return df_conts
df_conts = tile_with_noise(cycles)

In [ ]:
# cyc = cycles[0].iloc[:500]
# axis = plots(get_interesting_columns(cyc), title=str(idx));
# for ax in axis.flatten():
#     ax.axvline(idx)

In [ ]:
%%time
def flatten_and_cat(conts, deps, cats=None):
    cyc_cont = pd.DataFrame([o.values.flatten() for o in conts])
    
    # add dependent variables
    cyc_cont = pd.concat([cyc_cont, deps], axis=1, copy=False)
    cyc_cont.columns = [str(o) for o in range(cyc_cont.shape[1]-1)] + [dep_var]
    print('cyc_cont.shape', cyc_cont.shape)
    return cyc_cont
# deps = pd.DataFrame({'桩号': [o['桩号'].iloc[0] for o in cycles]})
deps = fmtr.get_y(is_problem1=True).iloc[:,0]
deps = pd.Series(np.tile(deps.values, mulr), name=deps.name)
cyc_cont = flatten_and_cat(df_conts, deps)

In [ ]:
cyc_cont.to_feather('tmp/cyc_cont_all_5c')

In [ ]:
cyc_cont = feather.read_dataframe('tmp/cyc_cont_all')

In [ ]:
# stacked = pd.concat([cyc_cont.iloc[int(valid_ratio * num_cycles):]]*mulr)
stacked = cyc_cont
print(stacked.shape)
valid_idx_tile = (valid_idx[:,None] + np.arange(mulr) * num_cycles).flatten() # take from all tiles
procs = Normalize
databunch = TabularDataBunch.from_df('tmp', stacked, dep_var, valid_idx=valid_idx, bs=bs, procs=procs)
learner = tabular_learner(databunch, [1000, 500])
learner.callback_fns.append(ShowGraph)
learner.model

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(3, 1e-1/5)
learner.save('fc4.0')

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(1, 1e-3)
learner.save('fc4.1')

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(3, 1e-3)
learner.save('fc4.2')

In [ ]:
learner.fit_one_cycle(5, 1e-3)
learner.save('fc4.3')

In [ ]:
x, y = next(iter(databunch.valid_dl))
y = to_np(y)
pred = to_np(learner.pred_batch(x)).squeeze()

In [ ]:
r = y.max() - y.min()
list(zip(pred, y, [(a-b)/(r)*100 for a,b in zip(pred, y)]))

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(5, 1e-5/2)
learner.save('fc3.4')